In [12]:
from pathlib import Path
from tokenizers import ByteLevelBPETokenizer
from transformers import RobertaTokenizerFast,RobertaForSequenceClassification,TrainingArguments,Trainer
import torch
from torch.utils.data import Dataset
import math
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [2]:
tokenizer = RobertaTokenizerFast.from_pretrained("./cornModel", max_len=512)
model = RobertaForSequenceClassification.from_pretrained('./cornModel')

Some weights of the model checkpoint at ./cornModel were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at ./cornModel and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.we

In [6]:
def read_data(filepath):
    filepath = Path(filepath)
    seqs = []
    exps = []
    lines = filepath.read_text(encoding="utf-8").splitlines()

    for line in lines:
            ele=line.split("\t")
            seqs.append(ele[1])
            exps.append(math.log10(float(ele[2])+1))    
    return seqs, exps

In [7]:
train_seq, train_exp = read_data('data/train.table.txt')
test_seq, test_exp = read_data('data/test.table.txt')

In [8]:
train_encodings = tokenizer(train_seq, truncation=True, padding=True)
test_encodings = tokenizer(test_seq, truncation=True, padding=True)

In [18]:
len(train_encodings)

2

In [21]:
class cornDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
        
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [22]:
train_dataset = cornDataset(train_encodings, train_exp)
test_dataset = cornDataset(test_encodings, test_exp)

In [43]:
training_args = TrainingArguments(
    output_dir='./tune_results',          # output directory
    num_train_epochs=1,              # total number of training epochs
    per_device_train_batch_size=2,  # batch size per device during training
    per_device_eval_batch_size=2,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

In [44]:
class MultilabelTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss_fct = torch.nn.L1Loss()
        loss = loss_fct(logits.view(-1, self.model.config.num_labels),
                        labels.float().view(-1, self.model.config.num_labels))
        return (loss, outputs) if return_outputs else loss

trainer = MultilabelTrainer(
    model=model,                        
    args=training_args,                  
    train_dataset=train_dataset,         
    eval_dataset=test_dataset   
)

In [39]:
train_dataset

In [45]:
trainer.train()

/root/.local/lib/python3.6/site-packages/torch/nn/modules/loss.py:94: UserWarning: Using a target size (torch.Size([1, 2])) that is different to the input size (torch.Size([2, 2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/root/.local/lib/python3.6/site-packages/torch/functional.py:1277: UserWarning: torch.norm is deprecated and may be removed in a future PyTorch release. Use torch.linalg.norm instead.
  "torch.norm is deprecated and may be removed in a future PyTorch release. "


Step,Training Loss
10,0.476000
20,0.825000
30,0.734700
40,0.526500
50,0.509000
60,0.559600
70,0.656900
80,0.363800
90,0.485900
100,0.605900


/root/.local/lib/python3.6/site-packages/torch/nn/modules/loss.py:94: UserWarning: Using a target size (torch.Size([1, 2])) that is different to the input size (torch.Size([2, 2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/root/.local/lib/python3.6/site-packages/torch/functional.py:1277: UserWarning: torch.norm is deprecated and may be removed in a future PyTorch release. Use torch.linalg.norm instead.
  "torch.norm is deprecated and may be removed in a future PyTorch release. "
/root/.local/lib/python3.6/site-packages/torch/nn/modules/loss.py:94: UserWarning: Using a target size (torch.Size([1, 2])) that is different to the input size (torch.Size([2, 2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/root/.local/lib/python3.6/site-packages/torch/f

/root/.local/lib/python3.6/site-packages/torch/nn/modules/loss.py:94: UserWarning: Using a target size (torch.Size([1, 2])) that is different to the input size (torch.Size([2, 2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/root/.local/lib/python3.6/site-packages/torch/functional.py:1277: UserWarning: torch.norm is deprecated and may be removed in a future PyTorch release. Use torch.linalg.norm instead.
  "torch.norm is deprecated and may be removed in a future PyTorch release. "
/root/.local/lib/python3.6/site-packages/torch/nn/modules/loss.py:94: UserWarning: Using a target size (torch.Size([1, 2])) that is different to the input size (torch.Size([2, 2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/root/.local/lib/python3.6/site-packages/torch/f

/root/.local/lib/python3.6/site-packages/torch/nn/modules/loss.py:94: UserWarning: Using a target size (torch.Size([1, 2])) that is different to the input size (torch.Size([2, 2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/root/.local/lib/python3.6/site-packages/torch/functional.py:1277: UserWarning: torch.norm is deprecated and may be removed in a future PyTorch release. Use torch.linalg.norm instead.
  "torch.norm is deprecated and may be removed in a future PyTorch release. "


TrainOutput(global_step=15000, training_loss=0.5620388622601827, metrics={'train_runtime': 1976.216, 'train_samples_per_second': 7.59, 'total_flos': 1832633933400000.0, 'epoch': 1.0, 'init_mem_cpu_alloc_delta': 115398, 'init_mem_gpu_alloc_delta': 0, 'init_mem_cpu_peaked_delta': 992529, 'init_mem_gpu_peaked_delta': 543437824, 'train_mem_cpu_alloc_delta': 1095994, 'train_mem_gpu_alloc_delta': 575409152, 'train_mem_cpu_peaked_delta': 33110322, 'train_mem_gpu_peaked_delta': 216523776})

In [46]:
trainer.save_model("./tunedCornModel")

In [48]:
trainer.evaluate()

/root/.local/lib/python3.6/site-packages/torch/nn/modules/loss.py:94: UserWarning: Using a target size (torch.Size([1, 2])) that is different to the input size (torch.Size([2, 2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


{'eval_loss': 0.5668526291847229,
 'eval_runtime': 166.2181,
 'eval_samples_per_second': 80.785,
 'epoch': 1.0,
 'eval_mem_cpu_alloc_delta': 73451,
 'eval_mem_gpu_alloc_delta': 0,
 'eval_mem_cpu_peaked_delta': 656785,
 'eval_mem_gpu_peaked_delta': 270726656}